# LIGO Event Rate

Here, we estimate the LIGO observed merger rate, given a distribution of PBH binaries.

**To-Do:** 
   - Implement and check 'first remapping' calculation
   - Write and upload a .py module for perfoming the 'second remapping'
   - Write a generic function for calculating the merger rate as a function of (f, M_PBH), with and without halo
   - Compare the 'without Halo' results with the Kamionkowski paper


**References:** [arXiv:1602.03842](https://arxiv.org/pdf/1602.03842.pdf), [arXiv:1606.03939](https://arxiv.org/abs/1606.03939), [arXiv:1704.04628](https://arxiv.org/abs/1704.04628)

In [ ]:
%matplotlib inline

from __future__ import division

import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl

from tqdm import tqdm

#----- MATPLOTLIB paramaters ---------
mpl.rcParams.update({'font.size': 18,'font.family':'serif'})

mpl.rcParams['xtick.major.size'] = 7
mpl.rcParams['xtick.major.width'] = 1
mpl.rcParams['xtick.minor.size'] = 3
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.minor.size'] = 3
mpl.rcParams['ytick.minor.width'] = 1
#--------------------------------------

from scipy.interpolate import interp1d
from scipy.integrate import quad


import emcee
import Remapping as RM
reload(RM)

import Sampling
reload(Sampling)

### LIGO sensitivity as a function of redshift

Read in the time-volume sensitivity for LIGO, from Fig. 7 of [arXiv:1704.04628](https://arxiv.org/abs/1704.04628):

In [ ]:
sens_data_10Msun = np.loadtxt("data/LIGO_sensitivity_10.txt")
sens_data_20Msun = np.loadtxt("data/LIGO_sensitivity_20.txt")
sens_data_40Msun = np.loadtxt("data/LIGO_sensitivity_40.txt")

sens_data_100Msun = np.loadtxt("data/LIGO_sensitivity_100.txt")
sens_data_200Msun = np.loadtxt("data/LIGO_sensitivity_200.txt")
sens_data_300Msun = np.loadtxt("data/LIGO_sensitivity_300.txt")

Do some interpolation to estimate the sensitivity for $M_\mathrm{PBH} = 30\,M_\odot$:

In [ ]:
sens_10Msun = interp1d(sens_data_10Msun[:,0], sens_data_10Msun[:,1], bounds_error=False, fill_value=0.0)
sens_20Msun = interp1d(sens_data_20Msun[:,0], sens_data_20Msun[:,1], bounds_error=False, fill_value=0.0)
sens_40Msun = interp1d(sens_data_40Msun[:,0], sens_data_40Msun[:,1], bounds_error=False, fill_value=0.0)

sens_100Msun = interp1d(sens_data_100Msun[:,0], sens_data_100Msun[:,1], bounds_error=False, fill_value=0.0)
sens_200Msun = interp1d(sens_data_200Msun[:,0], sens_data_200Msun[:,1], bounds_error=False, fill_value=0.0)
sens_300Msun = interp1d(sens_data_300Msun[:,0], sens_data_300Msun[:,1], bounds_error=False, fill_value=0.0)

In [ ]:
zvals = np.linspace(0, 0.7, 100)

sensitivities = [sens_10Msun, sens_20Msun, sens_40Msun,\
                     sens_100Msun, sens_200Msun, sens_300Msun]
labels = [r"$10\,M_\odot$",r"$20\,M_\odot$",r"$40\,M_\odot$",\
              r"$100\,M_\odot$",r"$200\,M_\odot$",r"$300\,M_\odot$"]

pl.figure(figsize=(6.1,6))

for sens, lab in zip(sensitivities, labels):
    pl.plot(zvals, sens(zvals), linewidth=1.5, label=lab)
    
pl.xlim(0,0.75)
pl.ylim(0, 0.75)
    
pl.xlabel(r"$z$")
pl.ylabel(r"$\mathrm{d}\langle V T \rangle/\mathrm{d}z$ (Gpc$^3$ yr)")

pl.legend(loc='upper right', frameon=False)

pl.show()

### Redshift-time relation

The time since the Big Bang is given by:

$$t(z) = \int_z^\infty \frac{\mathrm{d}z'}{(1+z') H(z')}\,,$$

where

$$H(z') = \sqrt{\Omega_\Lambda + \Omega_m (1+z)^3}\,,$$

in a flat universe. We use the Planck 2015 values from [arXiv:1502.01589](https://arxiv.org/abs/1502.01589), noting that there is some tension between different measurements of $H_0$:

- $H_0 = 67.8 \,\,\mathrm{km/s}\,\mathrm{Mpc}^{-1}$
- $\Omega_\Lambda = 0.692$
- $\Omega_m = 0.308$

In [ ]:
#Get H in units of 1/yr
#(km/Mpc) = 3.24e-20
H0_peryr = 67.8*(3.24e-20)*(60*60*24*365)

Omega_L = 0.692
Omega_m = 0.308

def Hubble(z):
    return H0_peryr*np.sqrt(Omega_L + Omega_m*(1+z)**3)

In [ ]:
#Calculate time from Big Bang until redshift z (in years)
def t_univ(z):
    integ = lambda x: 1.0/((1+x)*Hubble(x))
    return quad(integ, z, np.inf)[0]

In [ ]:
#Plot age of the universe as a function of Redshift

zlist = np.linspace(0, 2, 50)
tvals = np.asarray([t_univ(z) for z in zlist])

pl.figure(figsize=(6.1,6))
pl.plot(zlist, tvals/1e9)

pl.xlabel(r"Redshift, $z$")
pl.ylabel(r"Time since Big Bang, $t$ [Gyr]")

pl.ylim(0, 15)

pl.show()

z_of_t = interp1d(tvals, zlist)

In [ ]:
#Now also plot z as a function of time

pl.figure(figsize=(6.1,6))
pl.plot(tvals/1e9,zlist)

pl.axhline(0.6, linestyle='--', color='k')

pl.ylabel(r"Redshift, $z$")
pl.xlabel(r"Time since Big Bang, $t$ [Gyr]")

pl.show()

### Merger Rate

Calculate Merger rate from PBH binaries, as a function of the merger time. This is given by:

$$ R_\mathrm{merge}(t_\mathrm{merge}) = n_\mathrm{PBH} P(t_\mathrm{merge})\,.$$

Here, $n_\mathrm{PBH}$ is the number density of PBHs and $P(t_\mathrm{merge})$ is the PDF for the merger time of the binaries. Note that not all PBHs will form binaries, so we choose to normalise $P(t_\mathrm{merge})$ to the number of binaries which form *per PBH*:

$$\int P(t_\mathrm{merge}) \,\mathrm{d}t_\mathrm{merge} = \frac{n_\mathrm{binaries}}{n_\mathrm{PBH}}\,.$$

From this, the maximum normalisation is obtained when all PBHs form binaries, i.e. $n_\mathrm{binaries}/n_\mathrm{PBH} = 1/2$.

In [ ]:
#Calculate density of PBH binaries

#Fraction of DM made of PBHs
#f = 4e-3
#M_PBH = 20.0 #Solar masses

#Some cosmological parameters
G_N = 4.302e-3 #(pc/solar mass) (km/s)^2

h = 0.678
H0 = 100.0*h #(km/s) Mpc^-1
Omega_DM = 0.1186/(h**2)
    
def calc_nPBH(f, M_PBH):


    Omega_PBH = f*Omega_DM
    rho = 3.0*H0**2/(8.0*np.pi*(G_N*1e-6)) #Solar masses per Mpc^3

    n_PBH = (1e3)**3*rho*Omega_PBH/M_PBH #PBH per Gpc^3
    
    return n_PBH


In [ ]:
### Define + calculate merger time

#Variance of DM density perturbations at equality
sigma_eq = 0.005

#Volume factor for converting from Sasaki to Kamionkowski
volfac = (3.0/(4.0*np.pi))**(1.0/3.0)
alpha = 0.1

lambda_max = 3.0

def calcj(e):
    return np.sqrt(1-e**2)

#Mean PBH separation at z_eq (in pc)
rho_eq = 1512.0 #Solar masses per pc^3
def xbar_S(f, M_PBH):
    return 0.28*((M_PBH/30.0)**(1.0/3.0))*(f**(-1.0/3.0))

def xbar(f, M_PBH):
    return (3.0*M_PBH/(4*np.pi*rho_eq*(0.85*f)))**(1.0/3.0)

#Truncation radius of UCMH at z = z_eq:
def r_eq(M_PBH=30.0):
    return 1.96e-2*(M_PBH/30.0)**(1.0/3.0)

def M_halo(a, M_PBH=30.0):
    return M_PBH*(rtr_interp(a)/r_eq(M_PBH))**1.5

def density(r):
    if (r > r_cut):
        return 0
    else:
        return A*(r/r_tr)**(-3.0/2.0)
    
def Menc(r):
    if (r > r_cut):
        return M_PBH*(1+(r_cut/r_tr)**(3/2))
    else:
        return M_PBH*(1+(r/r_tr)**(3/2))    

def bigX(x, f, M_PBH):
    return (x/(xbar(f,M_PBH)))**3.0

def a_of_x(x, f, M_PBH):
    
    xb = xbar(f, M_PBH)
    return (alpha/(0.85*f))*x**4/xb**3

def x_of_a(a, f, M_PBH, withHalo = False):
    
    xb = xbar(f, M_PBH)
    
    if (not withHalo):
        
        return (a*f*0.85*xb**3/alpha)**(1.0/4.0)
    
    elif (withHalo):
                                                              
        xb_rescaled = xb * ((M_PBH + M_halo(a, M_PBH))/M_PBH )**(1./3.)            
        return (a*f*0.85*xb_rescaled**3/alpha)**(1.0/4.0)
 
#Distribution of j
def j_X(x, f, M_PBH):
    return bigX(x, f, M_PBH)*0.5*(1+sigma_eq**2/(0.85*f)**2)**0.5

def P_j(j, x, f, M_PBH):
    y = j/j_X(x, f, M_PBH)
    return (y**2/(1+y**2)**(3.0/2.0))/j

#Maximum semi-major axis
def a_max(f, M_PBH):
    return alpha*xbar(f, M_PBH)*(f*0.85)**(1.0/3.0)*((lambda_max)**(4.0/3.0))



def P_a_j(a, j, f=1e-2, M_PBH=30.0):
    xval = x_of_a(a, f, M_PBH)
    X = bigX(xval, f, M_PBH)
    xb = xbar(f, M_PBH)
    measure = (3.0/4.0)*(a**-0.25)*(0.85*f/(alpha*xb))**0.75
    return P_j(j, xval, f, M_PBH)*np.exp(-X)*measure

P_a_j_vec = np.vectorize(P_a_j, excluded=(2,3))

def P_a_j_withHalo(a, j, f=1e-2, M_PBH=30.0):
    xval = x_of_a(a, f, M_PBH, withHalo = True)
    X = bigX(xval, f, M_PBH)
    xb = xbar(f, M_PBH)
    
    rm = ((M_PBH + M_halo(a, M_PBH))/M_PBH)
    
    measure = (3.0/4.0)*(a**-0.25)*(0.85*f/(alpha*xb))**0.75
    return P_j(j, xval, f, M_PBH)*np.exp(-X)*measure*rm**(3./4.)

def P_a(a, f, M_PBH):
    xval = x_of_a(a, f, M_PBH)
    X = bigX(xval, f, M_PBH)
    xb = xbar(f, M_PBH)
    measure = (3.0/4.0)*(a**-0.25)*(0.85*f/(alpha*xb))**0.75
    return np.exp(-X)*measure


def P_a_withHalo(a, f, M_PBH):
    xval = x_of_a(a, f, M_PBH, withHalo = True)
    X = bigX(xval, f, M_PBH)
    xb = xbar(f, M_PBH)
    measure = (3.0/4.0)*(a**-0.25)*(0.85*f/(alpha*xb))**0.75
    return np.exp(-X)*measure

def P_la_lj(la,lj,f=1e-2, M_PBH=30.0):
    j = 10**lj
    a = 10**la
    return P_a_j(a, j, f, M_PBH)*a*j*(np.log(10)**2)

def P_la_lj_withHalo(la, lj, f=1e-3, M_PBH=30.0):
    j = 10**lj
    a = 10**la
    return P_a_j_withHalo(a, j, f, M_PBH)*a*j*(np.log(10)**2)

def t_coal(a, e, M_PBH=30.0):
    Q = (3.0/170.0)*(G_N*M_PBH)**-3 # s^6 pc^-3 km^-6
    tc = Q*a**4*(1-e**2)**(7.0/2.0) #s^6 pc km^-6
    tc *= 3.086e+13 #s^6 km^-5
    tc *= (3e5)**5 #s
    return tc/(60*60*24*365) #in years

def j_coal(a, t, M_PBH=30.0):
    Q = (3.0/170.0)*(G_N*M_PBH)**-3 # s^6 pc^-3 km^-6
    tc = t*(60*60*24*365)
    tc /= (3e5)**5
    tc /= 3.086e+13
    return (tc/(Q*a**4))**(1.0/7.0)


In [ ]:
def P_t(t, f, M_PBH, withHalo = False):
    amin = 1e-6
    if (withHalo):
        amax = a_max(f, 2.0*M_PBH)
    else:
        amax = a_max(f, M_PBH)

    alist = np.logspace(np.log10(amin), np.log10(amax), 100)
    Plist = 0.0*alist
    
    
    
    for i, a in enumerate(alist):
        M_tot = 1.0*M_PBH
        #if (withHalo):
        #    M_tot += M_halo(a, M_PBH)
        
        j = j_coal(a, t, M_tot)
        if (j >= 1):
            Plist[i] = 0
            
        else:
            e = np.sqrt(1-j**2)
            djde = e/j
            dedt = (j**2/e)/(7.0*t)
            
            if (withHalo):
                Plist[i] = P_a_j_withHalo(a, j, f, M_PBH)*djde*dedt
            else:
                Plist[i] = P_a_j(a, j, f, M_PBH)*djde*dedt
        
    #pl.figure()
    #pl.loglog(alist, Plist)
    #pl.show()
    
    return np.trapz(Plist, alist)

In [ ]:
def merger_rate(t_merge, f, M_PBH, withHalo=False):
    return calc_nPBH(f, M_PBH)*P_t(t_merge, f, M_PBH, withHalo) #Mergers per Gpc^3 per year

### PDFs

In [ ]:
f_ref = 1e-2
M_PBH = 30

a_list = np.logspace(np.log10(1e-6), np.log10(a_max(f=f_ref,M_PBH=2*30.0)),100)

j_list = np.asarray([j_coal(a, 13.5e9, M_PBH=30.0) for a in a_list])


P_a_j_vec = np.vectorize(P_a_j, excluded=(2,3))
P_a_j_withHalo_vec = np.vectorize(P_a_j_withHalo, excluded=(2,3))
P_list = P_a_j_vec(a_list, j_list, f_ref, M_PBH)
P_withHalo_list = P_a_j_withHalo_vec(a_list, j_list, f_ref, M_PBH)

a_peak = a_list[np.argmax(P_list)]

print j_coal(a_peak, 13.5e9, M_PBH)
print "Most likely value of a [pc]:", a_peak
print "Truncation radius [pc]:", rtr_interp(a_peak)
print "Req. eccentricity:", np.sqrt(1-(j_coal(a_peak, 13.5e9, M_PBH))**2)
print "Halo Mass [Msolar]:", M_halo(a_peak, M_PBH)

pl.figure()
pl.plot(a_list, P_list, label='Without Halo')
pl.plot(a_list, P_withHalo_list, label='With Halo')
pl.xlabel("Semi-major axis [pc]")
pl.ylabel(r"$P(a|t_\mathrm{merge} = t_\mathrm{univ})$ [a.u.]")

pl.axvline(a_max(f=f_ref,M_PBH=30.0), linestyle='--', color='k')

pl.legend()
pl.show()

In [ ]:
tvals = np.logspace(np.log10(1e9), np.log10(2e10))

pl.figure()
pl.plot(np.log10(tvals), tvals*np.log(10)*np.vectorize(P_t)(tvals, f, M_PBH, withHalo = False))
pl.ylim(0, 0.002)
pl.show()

pl.figure()
pl.plot(tvals, np.vectorize(P_t)(tvals, f, M_PBH, withHalo = False))
pl.show()

### LIGO Event Rate

Calculate number of events above threshold in the search presented in [arXiv:1602.03842](https://arxiv.org/pdf/1602.03842.pdf):

In [ ]:
#Calculate merger rate as a function of z
def merger_rate_z(z, f, M_PBH,withHalo=False):
    return merger_rate(t_univ(z), f, M_PBH, withHalo)


#Integrate over sensitivity to get number of events
#We don't have the sensitivity curve for 30 Msun, so do 20 Msun and 40 Msun
integrand_20 = lambda x: sens_20Msun(x)*merger_rate_z(x, f=1e-2, M_PBH=20.0)
integrand_40 = lambda x: sens_40Msun(x)*merger_rate_z(x, f=1e-2, M_PBH=20.0)

integrand_20_flat = lambda x: sens_20Msun(x)
integrand_40_flat = lambda x: sens_40Msun(x)


#Number of merger events above a given threshold in the search presented in 
N_20 = quad(integrand_20, 0, 0.7)[0]
N_40 = quad(integrand_40, 0, 0.7)[0]

N_20_flat = quad(integrand_20_flat, 0, 0.7)[0]
N_40_flat = quad(integrand_40_flat, 0, 0.7)[0]

print "Number of events above threshold:", N_20, "-", N_40


print "Estimated merger rate [Gpc^-3 yr^-1]:", N_20/N_20_flat, "-", N_40/N_40_flat


### Remapping

In [ ]:
f = 1e-2
M_PBH = 30

rtr_interp = RM.GetRtrInterp(M_PBH)

In [ ]:
samps = Sampling.GetSamples_MCMC(50000, P_la_lj_withHalo, 1e-6, a_max(f, 2.0*M_PBH), f, M_PBH)

In [ ]:
a_samps = np.asarray([10**x[0] for x in samps])
j_samps = np.asarray([10**x[1] for x in samps])
ti_samps = t_coal(a_samps,np.sqrt(1-j_samps**2), M_PBH)

af_samps = np.asarray([RM.calc_af(ai,M_PBH) for ai in a_samps])
jf_samps = np.asarray([RM.calc_jf(ji, ai,M_PBH) for ji,ai in zip(j_samps,a_samps)])
tf_samps = np.asarray([RM.calc_Tf(ti, ai, M_PBH) for ti, ai in zip(ti_samps, a_samps)])

#tf_samps = np.asarray([t_coal(10**x[0],np.sqrt(1-10**(2.0*x[1])), M_PBH) for x in samps])

In [ ]:
"""
xy = np.vstack([np.log10(a_samps),np.log10(j_samps)])
z = gaussian_kde(xy)(xy)

xy2 = np.vstack([np.log10(af_samps),np.log10(jf_samps)])
z2 = gaussian_kde(xy2)(xy2)
"""

pl.figure()

pl.scatter(np.log10(a_samps),np.log10(j_samps),alpha = 0.01)
pl.scatter(np.log10(af_samps),np.log10(jf_samps),alpha = 0.01)
#pl.scatter(np.log10(af_samps),np.log10(jf_samps), c=z2, s=100, edgecolor='', alpha= 0.1)
#pl.scatter(np.log10(a_samps), np.log10(j_samps), alpha=0.1)
#pl.scatter(np.log10(af_samps), np.log10(jf_samps), alpha=0.1)
#pl.scatter(np.log10(a_samps), np.log10(tf_samps), alpha=0.1)
#pl.colorbar()
pl.show()


In [ ]:
print np.min(ti_samps/1e9)
print np.min(tf_samps/1e9)

N_tot = 1.0*len(a_samps)

bins = np.logspace(7, 11, 101)
lbins = np.linspace(7, 11, 101)
#bins = np.linspace(1e7, 1e11, 1001)
bin_centres = np.sqrt(bins[:-1]*bins[1:])

pl.figure()
n_f, bins, patches  = pl.hist(np.log10(tf_samps), bins=lbins, normed=True, alpha=0.75)
n_i, bins, patches  = pl.hist(np.log10(ti_samps), bins=lbins, normed=True, alpha=0.75)


pl.axvline(np.log10(t_univ(0.0)))
pl.axvline(np.log10(t_univ(1.0)))

pl.xlim(9, np.log10(20e9))

pl.xlabel(r'$\log_{10}(t/\mathrm{yr})$')
pl.ylabel('Counts')

pl.show()


In [ ]:
#bin_centres = np.sqrt(bins[:-1]*bins[1:])

t1 = t_univ(0.7)
t2 = t_univ(0.0)
print t1/1e9,t2/1e9


tvals = np.logspace(8, 11, 1000)


P_true = np.asarray([P_t(t, f, M_PBH, withHalo=False) for t in tvals])
P_true_withHalo = np.asarray([P_t(t, f, M_PBH, withHalo=True) for t in tvals])

pl.figure()

pl.loglog(tvals, P_true, label="Without halo")
pl.loglog(tvals, P_true_withHalo, label="With halo")

pl.legend()
pl.show()

#P_norm = np.trapz(P_true*1.0/(bin_centres*np.log(10)), bin_centres)
P_norm = np.trapz(P_true, tvals)
print P_norm
P_norm_withHalo = np.trapz(P_true_withHalo, tvals)
print P_norm_withHalo

ni_normed = n_i/(bin_centres*np.log(10))
nf_normed = n_f/(bin_centres*np.log(10))

#norm = np.trapz(ni_normed, bin_centres)
norm = 1.0

ni_normed *= P_norm_withHalo/norm
nf_normed *= P_norm_withHalo/norm



pl.figure()
pl.step(bin_centres, ni_normed, where='mid', label='Initial')
pl.step(bin_centres, nf_normed, where='mid', label='Final')
#pl.loglog(bin_centres, (1.0/P_norm)*P_true*1.0/(bin_centres*np.log(10)), '+')
pl.plot(tvals, P_true, '-')

pl.xlabel(r"$t_\mathrm{merge} \, [yr]$")
pl.ylabel(r"$P(t_\mathrm{merge}) [\mathrm{yr}^{-1}]$")

pl.axvline(t1, linestyle='--', color='k')
pl.axvline(t2, linestyle='--', color='k')

pl.xlim(1e9, 50e9)

pl.legend(loc='best')

pl.show()

In [ ]:
#print t1, t2

P_ti_interp = interp1d(bin_centres, ni_normed, kind='linear')
P_tf_interp = interp1d(bin_centres, nf_normed, kind='linear')
#print quad(lambda y: P_lt_interp(y), 1.5e8, 0.9e11)[0]
#print quad(lambda y: P_lt_interp(y), t1, t2)[0]

def merger_rate_i(z, f, M_PBH):
    return P_ti_interp(t_univ(z))*calc_nPBH(f, M_PBH)

def merger_rate_f(z, f, M_PBH):
    return P_tf_interp(t_univ(z))*calc_nPBH(f, M_PBH)

#x = 0.5

#print "R_new/R_old = ", quad(lambda y: P_tf_interp(y), t1, t2)[0]/quad(lambda y: P_ti_interp(y), t1, t2)[0]

#Integrate over sensitivity to get number of events
#We don't have the sensitivity curve for 30 Msun, so do 20 Msun and 40 Msun

N_anal = np.zeros(2)
N_i = 0.0*N_anal
N_f = 0.0*N_anal
N_flat = 0.0*N_anal

for ind,sens_curve in enumerate([sens_20Msun, sens_40Msun]):
    
    integrand = lambda x: sens_curve(x)*merger_rate_z(x, f=1e-2, M_PBH=M_PBH)
    integrand_i = lambda x: sens_curve(x)*merger_rate_i(x, f=1e-2, M_PBH=M_PBH)
    integrand_f = lambda x: sens_curve(x)*merger_rate_f(x, f=1e-2, M_PBH=M_PBH)

    integrand_flat = lambda x: sens_curve(x)
    
    #Number of merger events above a given threshold in the search presented in 
    N_anal[ind] = quad(integrand, 0, 0.7)[0]
    N_i[ind] = quad(integrand_i, 0, 0.7)[0]
    N_f[ind] = quad(integrand_f, 0, 0.7)[0]
    #N_40 = quad(integrand_40, 0, 0.7)[0]
    N_flat[ind] = quad(integrand_flat, 0, 0.7)[0]

print "M_PBH = ", M_PBH, "; f = 1e-2"
print " Initial merger rate (analytic) [Gpc^-3 yr^-1]:", N_anal[0]/N_flat[0], "-", N_anal[1]/N_flat[1]
print " Initial merger rate (sampled) [Gpc^-3 yr^-1]:", N_i[0]/N_flat[0], "-", N_i[1]/N_flat[1]
print " Final merger rate [Gpc^-3 yr^-1]:",N_f[0]/N_flat[0], "-", N_f[1]/N_flat[1]


### Calculate unmapped merger rate

In [ ]:

fvals = np.logspace(-4, 0)

M_PBH = 30.0

Rvals = np.asarray([merger_rate_z(0,f, M_PBH) for f in fvals])

z_list = np.linspace(0, 0.5, 21)
merge_list = np.vectorize(merger_rate_z)(z_list, 1e-2, 30.0)

print np.trapz(merge_list, z_list)/(np.max(z_list) - np.min(z_list))

pl.figure(figsize=(7,5))

for M_PBH in [1, 30, 1000]:
    Rvals = np.asarray([merger_rate_z(0,f, M_PBH) for f in fvals])
    pl.loglog(fvals, Rvals, label='$'+str(M_PBH)+'\,M_\odot$')

pl.legend()

pl.xlabel('$f_\mathrm{PBH}$')
pl.ylabel('Merger Rate [$\mathrm{Gpc}^{-3}\,\mathrm{yr}^{-1}$]')
pl.tight_layout()

pl.savefig("Mergers_today.pdf")
pl.show()

### Automating the calculation of the merger rate today

 - Fix this so that I don't need rtr_interp to be global!
 - Speed up so that it doesn't take so damn long!
 
 **Note that $P(\log_{10}t)$ is reasonably flat, but $P(t)$ is actually not!** Need to be careful about what we mean by 'mergers today'...

Calculating the redshift as a function of t

In [ ]:
zlist = np.linspace(0, 2.0, 100)
tvals = np.asarray([t_univ(z) for z in zlist])

z_of_t_interp = interp1d(tvals, zlist)

def z_of_t(t):
    if ((t > t_univ(0.0)) or (t < t_univ(1.0))):
        return 5.0
    else:
        return z_of_t_interp(t)


In [ ]:
M_list = np.asarray([10., 20., 40.])
VT = np.zeros_like(M_list)

sensitivities = [sens_data_10Msun, sens_data_20Msun, sens_data_40Msun]

#Calculate stellar mass BH sensitivities by integrating
for i, sens in enumerate(sensitivities):
    VT[i] = np.trapz(sens[:,1], sens[:,0])

#Calculate IMBH sensitivities by converting from R90    
R_90_IMBH = np.loadtxt("data/LIGO_R90_IMBH.txt")
M_list = np.append(M_list, R_90_IMBH[:,0])
VT_IMBH = np.asarray([2.303/R_90_IMBH[j,1] for j in range(3)]) 

VT = np.append(VT, VT_IMBH)

#Do some plotting
pl.figure()
pl.loglog(M_list, VT, 's')
pl.xlabel(r"$M_\mathrm{BH} \,[M_\odot]$")
pl.ylabel(r"$\langle VT \rangle \, [\mathrm{Gpc}^3 \,\mathrm{yr}]$")
pl.title("Time-volume sensitivity", fontsize=14.0)
pl.show()

DOES THIS METHOD WORK???!?!

In [ ]:
def CalcLIGOMergerRate_unmapped(f, M_PBH, sens):
    global rtr_interp
    #Calculating truncation radius as a function of a
    rtr_interp = RM.GetRtrInterp(M_PBH)
    #print rtr_interp(0.01)
    
    #Generating samples from the PBH
    samps = Sampling.GetSamples_MCMC(1000, P_la_lj, 1e-6, a_max(f, M_PBH), f, M_PBH)
    
    #Calculating final merger time
    a_samps = np.asarray([10**x[0] for x in samps])
    j_samps = np.asarray([10**x[1] for x in samps])
    
    ti_samps = t_coal(a_samps,np.sqrt(1-j_samps**2), M_PBH)
    #tf_samps = np.asarray([RM.calc_Tf(ti, ai, M_PBH) for ti, ai in zip(ti_samps, a_samps)])

    N_samps = 1.0*len(ti_samps)
    
    #Calculate number of binaries merging in the LIGO range 'today'
    t1 = t_univ(1.0)
    t2 = t_univ(0.0)
    #print quad(sens, 0.0, 1.0)[0]
    #N_merge = np.sum( np.logical_and(tf_samps > t1,tf_samps < t2))
    N_merge = np.sum(sens(np.vectorize(z_of_t)(ti_samps)))/np.sum(ti_samps < t2)

    #print "   Estimate of normalisation of P(t):", x1
    
    #But we need to check the normalisation of P(t) in the range t1 -> t2
    
    #Normalise PDF correctly, to obtain fraction of BHs which form binaries in a given interval of t_merge
    tvals = np.logspace(9, 11, 500)
    P_true = np.asarray([P_t(t, f, M_PBH, withHalo=False) for t in tvals])
    f_bin = np.trapz(P_true, tvals)
    
    print f_bin
    frac = np.sum(ti_samps < t2)/N_samps

    #frac = 1.0
    z_upper = z_of_t(1e9)
    print z_upper
    
    frac = 1.0
    
    #Do the calculation in terms of z, to make sure my PDF is correctly normalised!
    zvals = np.linspace(0,z_upper,100)
    P_true_z = np.asarray([P_t(t_univ(z), f, M_PBH, withHalo=False) for z in zvals])
    f_bin_z = f_bin*np.trapz(P_true_z, zvals)/frac


    return f_bin_z*N_merge*calc_nPBH(f, M_PBH)/(quad(sens, 0.0, 1.0)[0])

In [ ]:
def CalcLIGOMergerRate_unmapped2(f, M_PBH, sens):
    global rtr_interp
    #Calculating truncation radius as a function of a
    rtr_interp = RM.GetRtrInterp(M_PBH)
    #print rtr_interp(0.01)
    
    #Generating samples from the PBH
    samps = Sampling.GetSamples_MCMC(10000, P_la_lj, 1e-6, a_max(f, M_PBH), f, M_PBH)
    
    #Calculating final merger time
    a_samps = np.asarray([10**x[0] for x in samps])
    j_samps = np.asarray([10**x[1] for x in samps])
    
    ti_samps = t_coal(a_samps,np.sqrt(1-j_samps**2), M_PBH)
    #tf_samps = np.asarray([RM.calc_Tf(ti, ai, M_PBH) for ti, ai in zip(ti_samps, a_samps)])

    N_samps = 1.0*len(ti_samps)
    print "   N_samps = ", N_samps
    
    #Calculate number of binaries merging in the LIGO range 'today'
    t1 = t_univ(1.0)
    t2 = t_univ(0.0)
    #print quad(sens, 0.0, 1.0)[0]
    #N_merge = np.sum( np.logical_and(tf_samps > t1,tf_samps < t2))
    t_crop = ti_samps[ti_samps < t2]
    z_samps = np.vectorize(z_of_t)(t_crop)
    N_merge = np.sum(Hubble(z_samps)*(1+z_samps)*sens(z_samps))/N_samps

    #print "   Estimate of normalisation of P(t):", x1
    
    #But we need to check the normalisation of P(t) in the range t1 -> t2
    
    #Normalise PDF correctly, to obtain fraction of BHs which form binaries in a given interval of t_merge
    tvals = np.logspace(9, 11, 500)
    P_true = np.asarray([P_t(t, f, M_PBH, withHalo=False) for t in tvals])
    f_bin = np.trapz(P_true, tvals)

    res = f_bin*N_merge*calc_nPBH(f, M_PBH)/(quad(sens, 0.0, 1.0)[0])
    
    return res

In [ ]:
def CalcLIGOMergerRate(f, M_PBH, sens):
    global rtr_interp
    #Calculating truncation radius as a function of a
    rtr_interp = RM.GetRtrInterp(M_PBH)
    #print rtr_interp(0.01)
    
    #Generating samples from the PBH
    samps = Sampling.GetSamples_MCMC(10000, P_la_lj_withHalo, 1e-6, a_max(f, 2*M_PBH), f, M_PBH)
    
    #Calculating final merger time
    a_samps = np.asarray([10**x[0] for x in samps])
    j_samps = np.asarray([10**x[1] for x in samps])
    
    ti_samps = t_coal(a_samps,np.sqrt(1-j_samps**2), M_PBH)
    tf_samps = np.asarray([RM.calc_Tf(ti, ai, M_PBH) for ti, ai in zip(ti_samps, a_samps)])

    N_samps = 1.0*len(ti_samps)
    print "   N_samps = ", N_samps
    
    #Calculate number of binaries merging in the LIGO range 'today'
    t1 = t_univ(1.0)
    t2 = t_univ(0.0)
    #print quad(sens, 0.0, 1.0)[0]
    #N_merge = np.sum( np.logical_and(tf_samps > t1,tf_samps < t2))
    t_crop = tf_samps[tf_samps < t2]
    z_samps = np.vectorize(z_of_t)(t_crop)
    N_merge = np.sum(Hubble(z_samps)*(1+z_samps)*sens(z_samps))/N_samps

    #print "   Estimate of normalisation of P(t):", x1
    
    #But we need to check the normalisation of P(t) in the range t1 -> t2
    
    #Normalise PDF correctly, to obtain fraction of BHs which form binaries in a given interval of t_merge
    tvals = np.logspace(9, 11, 500)
    P_true = np.asarray([P_t(t, f, M_PBH, withHalo=False) for t in tvals])
    f_bin = np.trapz(P_true, tvals)

    res = f_bin*N_merge*calc_nPBH(f, M_PBH)/(quad(sens, 0.0, 1.0)[0])
    
    return res

In [ ]:
#Mrate = np.zeros(10)
unmap = CalcLIGOMergerRate_unmapped2(1e-4, 300.0,sens_300Msun)
remap = CalcLIGOMergerRate(1e-4, 300.0,sens_300Msun)

print unmap, remap, (remap - unmap)/unmap

#for i in tqdm(range(10)):
#    Mrate[i] = CalcLIGOMergerRate_unmapped2(1e-2, 20.0,sens_20Msun)

In [ ]:
print merger_rate_z(0,1e-4, 300.0)

In [ ]:
pl.figure()
pl.hist(Mrate,50)
pl.show()

## Upper limit on f

In [ ]:
masses = [10.,20.,40.,100.,200.,300.]
sensitivities = [sens_10Msun, sens_20Msun, sens_40Msun,\
                     sens_100Msun, sens_200Msun, sens_300Msun]
R90_masses, R90_vals = np.loadtxt("data/R90_LIGO.txt", unpack=True)

f_UL = np.zeros_like(masses)

for m_ind in tqdm(range(6)):

    fvals = np.logspace(-4, 0, 11)

    Rvals = np.zeros_like(fvals)

    for i in range(len(fvals)):
        Rvals[i] = CalcLIGOMergerRate(fvals[i], masses[m_ind],sensitivities[m_ind])

    np.savetxt('data/LIGO_rate_' + str(int(masses[m_ind]))+'.txt', zip(fvals, Rvals), header='Merger rate for M_PBH/M_sun = '+str(masses[m_ind]) + '\nColumns: f_PBH, R [Gpc^-3 yr^-1]')

    f_inverse = interp1d(Rvals, fvals)

    R_upper = R90_vals[np.where(R90_masses==masses[m_ind])]

    print "90% UL is f = ", f_inverse(R_upper)

    f_UL[m_ind] = f_inverse(R_upper)
    
    pl.figure()
    pl.loglog(fvals, Rvals)

    pl.axhline(R_upper, linestyle='--', color='k')

    pl.title(r"$M_\mathrm{PBH} = " + str(masses[m_ind]) +"\,M_\odot$")
    pl.ylabel(r"$\mathcal{R}$ [$\mathrm{Gpc}^{-3}\,\mathrm{yr}^{-1}$]")
    pl.xlabel(r"$f_\mathrm{PBH}$")
    pl.tight_layout()

    pl.savefig('MergerRate_M=' + str(int(masses[m_ind])) + '.pdf')

    pl.show()
    

In [ ]:
np.savetxt('data/LIGO_limit_f.txt', zip(masses, f_UL))

### Plotting the LIGO upper limit

In [ ]:
#R90_LIGO = 2.303/VT

#np.savetxt("data/R90_LIGO.txt", zip(M_list, R90_LIGO), header="90% UL on merger rate density from LIGO\nColumns: M_BH [M_solar], R_90% [Gpc^-3 yr^-1]")

M1 = np.linspace(masses[0], masses[-1])

print f_UL

lf_interp = interp1d(np.log10(masses), np.log10(f_UL), kind='linear')

pl.figure(figsize=(7,5))
pl.fill_between(M1, 10**lf_interp(np.log10(M1)), 1e3, color='blue', alpha=0.20)
pl.loglog(masses, f_UL, 's', markersize=5.0)
pl.xlabel(r"$M_\mathrm{PBH} \,[M_\odot]$")
pl.ylabel(r"$f_\mathrm{PBH}$")
pl.title("PBH fraction upper limit", fontsize=14.0)

pl.ylim(1e-4, 1)
pl.xlim(1, 1e3)

pl.tight_layout()
pl.savefig('LIGO_UL.pdf')
pl.show()